In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)

In [3]:
# build a computational graph
y = w * x + b

In [4]:
# compute backward gradients
y.backward()

In [5]:
# print out the gradients

print(x.grad)
print(w.grad)
print(b.grad)

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



In [6]:
x = Variable(torch.randn(5,3))
y = Variable(torch.randn(5,2))

In [7]:
# build a linear layer
linear = nn.Linear(3, 2)
print("W: ", linear.weight)
print("b: ", linear.bias)

W:  Parameter containing:
 0.5322 -0.1513  0.3439
-0.1566 -0.4073  0.0902
[torch.FloatTensor of size 2x3]

b:  Parameter containing:
-0.4508
-0.0487
[torch.FloatTensor of size 2]



In [8]:
# build Loss and Optimizer

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [9]:
# forward propagation

pred = linear(x)

In [10]:
# compute loss
loss = criterion(pred, y)
print("loss: ", loss.data[0])

loss:  1.144765853881836


In [17]:
# Backpropagation

loss.backward()

In [18]:
# print out the gradients

print(linear.weight.grad)
print(linear.bias.grad)

Variable containing:
-0.2902 -0.5277  0.1719
 0.6956 -1.2329  0.1491
[torch.FloatTensor of size 2x3]

Variable containing:
-0.5276
 0.4790
[torch.FloatTensor of size 2]



In [19]:
# 1-step Optimization (gradient descent))

optimizer.step()

In [20]:
# print loss after optimization
pred = linear(x)
loss = criterion(pred, y)

In [21]:
print("loss after 1 step optimization: ", loss.data[0])

loss after 1 step optimization:  1.1158393621444702


In [25]:
train_dataset = dsets.CIFAR10(root="./datasets/",
                             train=True, transform=transforms.ToTensor(),
                             download=False)

In [26]:
# select one data pair (read data from disk)
image, label = train_dataset[0]

In [30]:
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [31]:
# Data Loader (this provides queue and thread in a very simple way)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=2)

In [32]:
# When iteration starts, queue and thread start to load dataset from files.
data_iter = iter(train_loader)

In [33]:
# Mini-batch images and labels.
images, labels = data_iter.next()

In [34]:
# Actual usage of data loader is as below:

for images, labels in train_loader:
    # Training code
    pass

In [35]:
#===================== Input pipline for custom dataset =====================#
# You should build custom dataset as below.
class CustomDataset(data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file path or list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# Then, you can just use prebuilt torch's data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)


#========================== Using pretrained model ==========================#
# Download and load pretrained resnet.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only top layer of the model.
for param in resnet.parameters():
    param.requires_grad = False
    
# Replace top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is for example.

# For test.
images = Variable(torch.randn(10, 3, 256, 256))
outputs = resnet(images)
print (outputs.size())   # (10, 100)


#============================ Save and load the model ============================#
# Save and load the entire model.
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

# Save and load only the model parameters(recommended).
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\Chauh/.torch\models\resnet18-5c106cde.pth
4.3%


KeyboardInterrupt: 